In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, 
NavigationToolbar2Tk)
import numpy as np
import pvlib
import rdtools
%matplotlib inline
from rdtools.soiling import monthly_soiling_rates
from rdtools.soiling import annual_soiling_ratios
from pandastable import Table, TableModel, config

import tkinter as tk
from tkinter import *
from tkinter import ttk
from tkinter import simpledialog
from tkinter.messagebox import showinfo
# from tkinter.tix import *
from PIL import Image, ImageTk

import pathlib
import os
import shutil

import openpyxl as xl
from openpyxl import load_workbook
import re

import xlwings as xw

In [ ]:
# Set the random seed for numpy to ensure consistent results
np.random.seed(0)

# Importer les données et premier graphique

In [ ]:
# Import the example data
def Importer(self, parent, controller, conditional):

#     if counter == 0:
#         counter = counter + 1
    print(conditional)  
    global df
    #df = pd.read_excel("C:/Users/quentin.fiessinger/Documents/donnees_gaillac.xlsx", index_col=0, parse_dates=True)
    finalpath = filepath.get().replace("\\", "/")
    finalpath = finalpath.replace("\"", "")
        
    df = pd.read_excel(finalpath, index_col=0, parse_dates=True)       
    
    df = df.rename(columns = {
    'ambient_temp': 'Tamb',
    'poa_irradiance': 'poa',
    'Eedf': 'ac_power',
    })

    # Specify the Metadata
    global meta
    meta = {"latitude": float(latitude.get()),
    "longitude": float(longitude.get()),
    "timezone": str(timezone.get()),
    "gamma_pdc": float(gamma_pdc.get()),
    "azimuth": int(azimuth.get()),
    "tilt": int(tilt.get()),
    "power_dc_rated": float(power_dc_rated.get()),
    "temp_model_params":
    pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm'][str(temp_model_params.get())]}

    df.index = df.index.tz_localize(meta['timezone'])

    # There is some missing data, but we can infer the frequency from
    # the first several data points
    freq = pd.infer_freq(df.index[:10])

    # Then set the frequency of the dataframe.
    # It is recommended not to up- or downsample at this step
    # but rather to use interpolate to regularize the time series
    # to its dominant or underlying frequency. Interpolate is not
    # generally recommended for downsampling in this application.
    df = rdtools.interpolate(df, freq)

    # Calculate cell temperature
    df['Tcell'] = pvlib.temperature.sapm_cell(df.poa, df.Tamb,
                                      df.wind_speed, **meta['temp_model_params'])

    # plot the AC power time series
    # Trace le graphique de la série temporelle de la puissance AC 
    # fig, ax = plt.subplots(figsize=(4,3))
    fig, ax = plt.subplots()
    ax.plot(df.index, df.ac_power, 'o', alpha=0.01)
    ax.set_ylim(0, int(puissance.get()))
    plt.title('Puissance produite par rapport à la date')
    fig.autofmt_xdate()
    ax.set_ylabel('AC Power (W)');
    
    canvas = FigureCanvasTkAgg(fig, master = self)
    canvas.draw()
    
    #canvas.get_tk_widget().grid(row = 4, column = 4)
    
    #toolbar = NavigationToolbar2Tk(canvas, self).grid(row = 6, column = 10)
    #toolbar.update()
    
    canvas.get_tk_widget().place(x = 350, y = 50)

# Normalisation

In [ ]:
def Normalisation(self, parent, controller, conditional):
    
#     if counter == 1:
#         counter = counter + 1
    print(conditional)  
        
    df['power'] = df['ac_power']

    # Calculate the expected power with a simple PVWatts DC model
    modeled_power = pvlib.pvsystem.pvwatts_dc(df['poa'], df['Tcell'], meta['power_dc_rated'],
                                              meta['gamma_pdc'], 25.0 )

    # Calculate the normalization, the function also returns the relevant insolation for
    # each point in the normalized PV energy timeseries
    normalized, insolation = rdtools.normalize_with_expected_power(df['power'],
                                                                   modeled_power,
                                                                   df['poa'])

    df['normalized'] = normalized
    df['insolation'] = insolation

    # Plot the normalized power time series
    # Trace la série temporelle de la puissance normalisée
    fig, ax = plt.subplots()
    ax.plot(normalized.index, normalized, 'o', alpha = 0.05)
    ax.set_ylim(0,2)
    fig.autofmt_xdate()
    ax.set_ylabel('Normalized energy');
    
    canvas = FigureCanvasTkAgg(fig, master = self)
    canvas.draw()
    
    #canvas.get_tk_widget().grid(row = 4, column = 4)
    
    #toolbar = NavigationToolbar2Tk(canvas, self).grid(row = 6, column = 10)
    #toolbar.update()
    
    canvas.get_tk_widget().place(x = 350, y = 50)

# Filtrage

In [ ]:
def Filtrage(self, parent, controller, conditional):
#     if counter == 2:
#         counter = counter + 1
    print(conditional)  
    
    # Calculate a collection of boolean masks that can be used
    # to filter the time series
    normalized_mask = rdtools.normalized_filter(df['normalized'])
    poa_mask = rdtools.poa_filter(df['poa'])
    tcell_mask = rdtools.tcell_filter(df['Tcell'])
    # Note: This clipping mask may be disabled when you are sure the system is not 
    # experiencing clipping due to high DC/AC ratio
    global clip_mask
    clip_mask = rdtools.clip_filter(df['power'], 'quantile')

    # filter the time series and keep only the columns needed for the
    # remaining steps
    global filtered
    filtered = df[normalized_mask & poa_mask & tcell_mask & clip_mask]
    filtered = filtered[['insolation', 'normalized']]

    fig, ax = plt.subplots()
    ax.plot(filtered.index, filtered.normalized, 'o', alpha = 0.05)
    ax.set_ylim(0,2)
    fig.autofmt_xdate()
    ax.set_ylabel('Normalized energy');
    
    canvas = FigureCanvasTkAgg(fig, master = self)
    canvas.draw()
    
    #canvas.get_tk_widget().grid(row = 4, column = 4)
    
    #toolbar = NavigationToolbar2Tk(canvas, self).grid(row = 6, column = 10)
    #toolbar.update()
    
    canvas.get_tk_widget().place(x = 350, y = 50)

# Rassemblement

In [ ]:
def Rassemblement(self, parent, controller, conditional):
#     if counter == 3:
#         counter = counter + 1
    print(conditional)  
    global daily
    daily = rdtools.aggregation_insol(filtered.normalized, filtered.insolation,
                                      frequency = 'D')

    fig, ax = plt.subplots()
    ax.plot(daily.index, daily, 'o', alpha = 0.1)
    ax.set_ylim(0,2)
    fig.autofmt_xdate()
    ax.set_ylabel('Normalized energy');
    
    canvas = FigureCanvasTkAgg(fig, master = self)
    canvas.draw()
    
    #canvas.get_tk_widget().grid(row = 4, column = 4)
    
    #toolbar = NavigationToolbar2Tk(canvas, self).grid(row = 6, column = 10)
    #toolbar.update()
    
    canvas.get_tk_widget().place(x = 350, y = 50)

# Calcul de la dégradation

In [ ]:
def Degradation(self, parent, controller, conditional, intervalDeg):
#     if counter == 4:
#         counter = counter + 1
    print(conditional)  
    # Calculate the degradation rate using the YoY method
    yoy_rd, yoy_ci, yoy_info = rdtools.degradation_year_on_year(daily, confidence_level= float(intervalDeg))
    # Note the default confidence_level of 68.2 is appropriate if you would like to 
    # report a confidence interval analogous to the standard deviation of a normal
    # distribution. The size of the confidence interval is adjustable by setting the
    # confidence_level variable.

    # Visualize the results

    degradation_fig = rdtools.degradation_summary_plots(
        yoy_rd, yoy_ci, yoy_info, daily,
        summary_title='Sensor-based degradation results',
        scatter_ymin=0.5, scatter_ymax=1.1,
        hist_xmin=-30, hist_xmax=45, bins=100
    )
    
    
    
    canvas = FigureCanvasTkAgg(degradation_fig, master = self)
    canvas.draw()
    
    #canvas.get_tk_widget().grid(row = 4, column = 4)
    
    #toolbar = NavigationToolbar2Tk(canvas, self).grid(row = 6, column = 10)
    #toolbar.update()
    
    canvas.get_tk_widget().place(x = 250, y = 150)
    
    
    ttk.Label(self, text = 'The P95 exceedance level is %.2f%%/yr' % yoy_info['exceedance_level']).place(x = 500, y = 30)
    
    ttk.Label(self, text = str(np.round(yoy_rd, 3))).place(x = 500, y = 50)
              
    ttk.Label(self, text = str(np.round(yoy_ci, 3))).place(x = 500, y = 70)

# Calcul d'encrassement

In [ ]:
def Encrassement(self, parent, controller, Page6, conditional, num, intervalEnc):
#     if counter == 5:
#         counter = counter + 1
    print(conditional)  
    # Calculate the daily insolation, required for the SRR calculation
    daily_insolation = filtered['insolation'].resample('D').sum()

    # Perform the SRR calculation
    from rdtools.soiling import soiling_srr
    cl = int(intervalEnc)
    sr, sr_ci, soiling_info = soiling_srr(daily, daily_insolation,
                                          confidence_level=cl)

    fig = rdtools.plotting.soiling_monte_carlo_plot(soiling_info, daily, profiles=200);
    
    label = ttk.Label(self, text = "Plot Monte Carlo realizations of soiling profiles")
    label.place(x = 400, y = 120)
    canvas = FigureCanvasTkAgg(fig, master = self)
    canvas.draw()
    canvas.get_tk_widget().place(x = 400, y = 150)

    fig2 = rdtools.plotting.soiling_interval_plot(soiling_info, daily);

    label2 = ttk.Label(self, text = "Plot the slopes for \"valid\" soiling intervals identified, assuming perfect cleaning events")
    label2.place(x = 800, y = 120)
    canvas2 = FigureCanvasTkAgg(fig2, master = self)
    canvas2.draw()
    canvas2.get_tk_widget().place(x = 800, y = 150)
    
    fig3 = rdtools.plotting.soiling_rate_histogram(soiling_info, bins = int(num))

    label3 = ttk.Label(self, text = "View a histogram of the valid soiling rates found for the data set")
    label3.place(x = 1300, y = 120)
    canvas3 = FigureCanvasTkAgg(fig3, master = self)
    canvas3.draw()
    canvas3.get_tk_widget().place(x = 1200, y = 150)
    
    ttk.Label(self, text = 'The P50 insolation-weighted soiling ratio is %0.3f'%sr).place(x = 500, y = 50)

    ttk.Label(self, text = 'The %0.1f confidence interval for the insolation-weighted'
          ' soiling ratio is %0.3f–%0.3f'%(cl, sr_ci[0], sr_ci[1])).place(x = 500, y = 50)
    
    label4 = ttk.Label(self, text = "Calculate and view a monthly soiling rate summary")
    label4.place(x = 900, y = 700)
    tv1 = ttk.Treeview(self)
    tv1.place(x = 900, y = 720)
    
    treescrolly = tk.Scrollbar(self, orient = "vertical", command = tv1.yview)
    treescrollx = tk.Scrollbar(self, orient = "horizontal", command = tv1.xview)
    tv1.configure(xscrollcommand = treescrollx.set, yscrollcommand = treescrolly.set)
#     treescrollx.pack(side = "bottom", fill = "x")
#     treescrolly.pack(side = "right", fill = "y")
#     treescrollx.place(x = 150, y = 800)
#     treescrolly.place(x = 150, y = 400)
    
    df = monthly_soiling_rates(soiling_info['soiling_interval_summary'], confidence_level=cl)
    
    tv1["column"] = list(df.columns)
    tv1["show"] = "headings"
    
    for column in tv1["columns"]:
        tv1.heading(column, text = column)
        
    df_rows = df.to_numpy().tolist()
    for row in df_rows:
        tv1.insert("", "end", values = row)
    
    label5 = ttk.Label(self, text = "Calculate and view annual insolation-weighted soiling ratios and their confidence intervals based on the Monte Carlo simulation.")
    label5.place(x = 15, y = 700)
    tv2 = ttk.Treeview(self)
    tv2.place(x = 15, y = 720)
    
    treescrolly2 = tk.Scrollbar(self, orient = "vertical", command = tv2.yview)
    treescrollx2 = tk.Scrollbar(self, orient = "horizontal", command = tv2.xview)
    tv2.configure(xscrollcommand = treescrollx2.set, yscrollcommand = treescrolly2.set)
#     treescrollx.pack(side = "bottom", fill = "x")
#     treescrolly.pack(side = "right", fill = "y")
#     treescrollx.place(x = 150, y = 800)
#     treescrolly.place(x = 150, y = 400)
    
    df2 = annual_soiling_ratios(soiling_info['stochastic_soiling_profiles'], daily_insolation, confidence_level=cl)
    
    tv2["column"] = list(df2.columns)
    tv2["show"] = "headings"
    
    for column in tv2["columns"]:
        tv2.heading(column, text = column)
        
    df2_rows = df2.to_numpy().tolist()
    for row in df2_rows:
        tv2.insert("", "end", values = row)
    
    label6 = ttk.Label(self, text = "View the first several rows of the soiling interval summary table")
    label6.place(x = 50, y = 445)
    tv3 = ttk.Treeview(self)
    tv3.place(x = 50, y = 465)
    
    treescrolly3 = tk.Scrollbar(self, orient = "vertical", command = tv3.yview)
    treescrollx3 = tk.Scrollbar(self, orient = "horizontal", command = tv3.xview)
    tv3.configure(xscrollcommand = treescrollx3.set, yscrollcommand = treescrolly3.set)
#     treescrollx.pack(side = "bottom", fill = "x")
#     treescrolly.pack(side = "right", fill = "y")
#     treescrollx.place(x = 150, y = 800)
#     treescrolly.place(x = 150, y = 400)
    
    df3 = soiling_info['soiling_interval_summary']
    
    tv3["column"] = list(df3.columns)
    tv3["show"] = "headings"
    
    for column in tv3["columns"]:
        tv3.heading(column, text = column)
        
    df3_rows = df3.to_numpy().tolist()
    for row in df3_rows:
        tv3.insert("", "end", values = row)  
    

    clair0()
    clair1()
    clair2()
    clair3()
    clair4()

# Modèle ciel clair

# Ciel clair 0 : Calculs préliminaires

In [ ]:
# Calculate the clear sky POA irradiance
def clair0():
    loc = pvlib.location.Location(meta['latitude'], meta['longitude'], tz = meta['timezone'])
    sun = loc.get_solarposition(df.index)
    clearsky = loc.get_clearsky(df.index, solar_position=sun)

    cs_irradiance = pvlib.irradiance.get_total_irradiance(meta['tilt'], meta['azimuth'], sun['apparent_zenith'],
                                                         sun['azimuth'], clearsky['dni'], clearsky['ghi'],
                                                         clearsky['dhi'])
    df['clearsky_poa'] = cs_irradiance.poa_global
    # Renormalize the clear sky POA irradiance
    df['clearsky_poa'] = rdtools.irradiance_rescale(df.poa, df.clearsky_poa,
                                                    method='iterative', convergence_threshold=4e-03)

    # Calculate the clearsky temperature
    df['clearsky_Tamb'] = rdtools.get_clearsky_tamb(df.index, meta['latitude'],
                                                    meta['longitude'])
    df['clearsky_Tcell'] = pvlib.temperature.sapm_cell(df.clearsky_poa, df.clearsky_Tamb,
                                                       0, **meta['temp_model_params'])

# Ciel Clair 1 : Normalisation

In [ ]:
# Calculate the expected power with a simple PVWatts DC model
def clair1():
    clearsky_modeled_power = pvlib.pvsystem.pvwatts_dc(df['clearsky_poa'],
                                                       df['clearsky_Tcell'],
                                                       meta['power_dc_rated'], meta['gamma_pdc'], 25.0 )

    # Calculate the normalization, the function also returns the relevant insolation for
    # each point in the normalized PV energy timeseries
    global clearsky_normalized
    global clearsky_insolation
    clearsky_normalized, clearsky_insolation = rdtools.normalize_with_expected_power(
        df['power'],
        clearsky_modeled_power,
        df['clearsky_poa']
    )

    df['clearsky_normalized'] = clearsky_normalized
    df['clearsky_insolation'] = clearsky_insolation

# Ciel clair 2 : Filtrage

In [ ]:
# Perform clearsky filter
def clair2():
    cs_normalized_mask = rdtools.normalized_filter(df['clearsky_normalized'])
    cs_poa_mask = rdtools.poa_filter(df['clearsky_poa'])
    cs_tcell_mask = rdtools.tcell_filter(df['clearsky_Tcell'])

    csi_mask = rdtools.csi_filter(df.insolation, df.clearsky_insolation)
    
    global clearsky_filtered
    clearsky_filtered = df[cs_normalized_mask & cs_poa_mask & cs_tcell_mask &
                           clip_mask & csi_mask]
    clearsky_filtered = clearsky_filtered[['clearsky_insolation', 'clearsky_normalized']]

# Ciel clair 3 : Rassemblement

In [ ]:
def clair3():
    global clearsky_daily
    clearsky_daily = rdtools.aggregation_insol(clearsky_filtered.clearsky_normalized,
                                               clearsky_filtered.clearsky_insolation)

# Ciel clair 4 : Calcul de la dégradation

In [ ]:
# Calculate the degradation rate using the YoY method
def clair4():
    cs_yoy_rd, cs_yoy_ci, cs_yoy_info = rdtools.degradation_year_on_year(
        clearsky_daily,
        confidence_level=68.2
    )

    # Note the default confidence_level of 68.2 is appropriate if you would like to 
    # report a confidence interval analogous to the standard deviation of a normal
    # distribution. The size of the confidence interval is adjustable by setting the
    # confidence_level variable.

    # Visualize the results
    clearsky_fig = rdtools.degradation_summary_plots(
        cs_yoy_rd, cs_yoy_ci, cs_yoy_info, clearsky_daily,
        summary_title='Clear-sky-based degradation results',
        scatter_ymin=0.5, scatter_ymax=1.1,
        hist_xmin=-30, hist_xmax=45, plot_color='orangered',
        bins=100);

    print('The P95 exceedance level with the clear sky analysis is %.2f%%/yr' %
          cs_yoy_info['exceedance_level'])

# Executable

In [ ]:
def erreurs(integer, self):
    if integer == 1:
        label2 = ttk.Label(self, text = "La donne entrer ne correspond pas au bon format ")
        label2.place(x = 900, y = 370)
        label2.after(10000, lambda: label2.destroy())
    if integer == 2:
        label3 = ttk.Label(self, text = "Utiliser un point au lieu d'une virgule")
        label3.place(x = 900, y = 400)
        label3.after(10000, lambda: label3.destroy())
    if integer == 3:
        label4 = ttk.Label(self, text = "Le symbole du degree n'est pas reconu ne pas l'utiliser")
        label4.place(x = 900, y = 400)
        label4.after(10000, lambda: label4.destroy())
    if integer == 4:
        label5 = ttk.Label(self, text = "N'oubliez pas de selection l'option en bonus")
        label5.place(x = 900, y = 400)
        label5.after(10000, lambda: label5.destroy())
    if integer == 5:
        label6 = ttk.Label(self, text = "Entrer le timezone en suivant le format de la tz database time zones")
        label6.place(x = 900, y = 370)
        label6.after(10000, lambda: label6.destroy())
    if integer == 6:
        label7 = ttk.Label(self, text = "Ce chemin d'access n'est pas reconnu")
        label7.place(x = 900, y = 370)
        label7.after(10000, lambda: label7.destroy())
    if integer == 7:
        label8 = ttk.Label(self, text = "Il manque des donnes")
        label8.place(x = 900, y = 370)
        label8.after(10000, lambda: label8.destroy())
    if integer == 8:
        label9 = ttk.Label(self, text = "https://en.wikipedia.org/wiki/List_of_tz_database_time_zones")
        label9.place(x = 900, y = 400)
        label9.after(10000, lambda: label9.destroy())
    if integer == 9:
        label10 = ttk.Label(self, text = "Pour la latitude entrer un nombre")
        label10.place(x = 900, y = 340)
        label10.after(10000, lambda: label10.destroy()) 
    if integer == 10:
        label11 = ttk.Label(self, text = "Pour la longitude entrer un nombre")
        label11.place(x = 900, y = 340)
        label11.after(10000, lambda: label11.destroy()) 
    if integer == 11:
        label12 = ttk.Label(self, text = "Le timezone cherche une combinaison de lettre et chiffre")
        label12.place(x = 900, y = 340)
        label12.after(10000, lambda: label12.destroy()) 
    if integer == 12:
        label13 = ttk.Label(self, text = "Pour le gamma pdc entrer un nombre")
        label13.place(x = 900, y = 340)
        label13.after(10000, lambda: label13.destroy())
    if integer == 13:
        label14 = ttk.Label(self, text = "Pour le azimuth entrer un nombre")
        label14.place(x = 900, y = 340)
        label14.after(10000, lambda: label14.destroy())
    if integer == 14:
        label15 = ttk.Label(self, text = "Pour le tilt entrer un nombre")
        label15.place(x = 900, y = 340)
        label15.after(10000, lambda: label15.destroy())
    if integer == 15:
        label16 = ttk.Label(self, text = "Un calcul necessaire pour la prochaine etape n'a pas ete accompli")
        label16.place(x = 900, y = 340)
        label16.after(10000, lambda: label16.destroy())
    if integer == 16:
        label17 = ttk.Label(self, text = "Pour le power dc entrer un nombre")
        label17.place(x = 900, y = 340)
        label17.after(10000, lambda: label17.destroy())
    if integer == 17:
        label18 = ttk.Label(self, text = "Pour la puissance maximal entrer un nombre")
        label18.place(x = 900, y = 340)
        label18.after(10000, lambda: label18.destroy())
    if integer == 18:
        label19 = ttk.Label(self, text = "Pour le bins entrer un nombre")
        label19.place(x = 900, y = 340)
        label19.after(10000, lambda: label19.destroy())
    if integer == 19:
        label20 = ttk.Label(self, text = "Pour l'interval de confiance entrer un nombre")
        label20.place(x = 900, y = 340)
        label20.after(10000, lambda: label20.destroy())

In [ ]:
LARGEFONT =("Verdana", 20)

class tkinterApp(tk.Tk):
     
    # __init__ function for class tkinterApp
    def __init__(self, *args, **kwargs):
         
        # __init__ function for class Tk
        tk.Tk.__init__(self, *args, **kwargs)
        

        # creating a container
        container = tk.Frame(self) 
        container.pack(side = "top", fill = "both", expand = True)
        self.geometry("1280x700")
  
        container.grid_rowconfigure(0, weight = 1)
        container.grid_columnconfigure(0, weight = 1)
  
        # initializing frames to an empty array
        self.frames = {} 
  
        # iterating through a tuple consisting
        # of the different page layouts
        for F in (StartPage, Page1, Page2, Page3, Page4, Page5, Page6):
  
            frame = F(container, self)
  
            # initializing frame of that object from
            # startpage, page1, page2 respectively with
            # for loop
            self.frames[F] = frame
  
            frame.grid(row = 0, column = 0, sticky ="nsew")
  
        self.show_frame(StartPage)
  
    # to display the current frame passed as
    # parameter
    def show_frame(self, cont):
        frame = self.frames[cont]
        frame.tkraise()
        
# first window frame startpage

class StartPage(tk.Frame):
    def __init__(self, parent, controller):
        
        tk.Frame.__init__(self, parent)
        
        # label of frame StartPage

        
#         sb = Scrollbar(
#             self,
#             orient = VERTICAL
#         )
        
#         sb.grid(row = 0, column = 5, sticky = NS)
        
#         self.config(yscrollcommand = sb.set)
#         sb.config(command = self.yview)
        
#         swin = Tk.ScrolledWindow(StartPage, width = 1280, height = 700)
#         swin.grid(row = 0, column = 0)
        
        global messages
        messages = StringVar()
        
        label = ttk.Label(self, text ="Startpage", font = LARGEFONT)
        label.grid(row = 0, column = 0, sticky = tk.W, pady = 10)
        
        StartPage.conditional = 0        
        StartPage.filepathtest = ""
        StartPage.num = 0
        StartPage.intervalDeg = 0
        StartPage.intervalEnc = 0
        
        global filepath
        file = ttk.Label(self, text = "Entrer le chemin d'access de votre fichier").place(x = 250, y = 70)
        filepath = ttk.Entry(self)
        filepath.place(x = 520, y = 70)
        

        
        global latitude
        lat = ttk.Label(self, text = "Entrer la latitude du projet").place(x = 250, y = 120)
        latitude = ttk.Entry(self)
        latitude.place(x = 520, y = 120)
        
        global longitude
        long = ttk.Label(self, text = "Entrer la longitude du projet").place(x = 250, y = 170)
        longitude = ttk.Entry(self)
        longitude.place(x = 520, y = 170)
        
        global timezone
        time = ttk.Label(self, text = "Entrer la timezone du projet").place(x = 250, y = 220)
        timezone = ttk.Entry(self)
        timezone.place(x = 520, y = 220)
        
        global gamma_pdc
        gamma = ttk.Label(self, text = "Entrer le gamma pdc du projet").place(x = 250, y = 270)
        gamma_pdc = ttk.Entry(self)
        gamma_pdc.place(x = 520, y = 270)

        global azimuth
        azimuth = ttk.Label(self, text = "Entrer le azimuth du projet").place(x = 250, y = 320)
        azimuth = ttk.Entry(self)
        azimuth.place(x = 520, y = 320)
        
        global tilt
        tilt = ttk.Label(self, text = "Entrer le tilt du projet").place(x = 250, y = 370)
        tilt = ttk.Entry(self)
        tilt.place(x = 520, y = 370)
        
        global power_dc_rated
        ttk.Label(self, text = "Entrer le power dc rated du projet").place(x = 250, y = 420)
        power_dc_rated = ttk.Entry(self)
        power_dc_rated.place(x = 520, y = 420)
        
        global temp_model_params
        temp = ttk.Label(self, text = "Entrer le parametre du model temporel du pojet").place(x = 250, y = 470)

        options = [
            "open_rack_glass_glass",
            "close_mount_glass_glass",
            "open_rack_glass_polymer",
            "insulated_back_glass_polymer"
        ]
    
        temp_model_params = StringVar()
        
        temp_model_params.set("Options")
    
        drop = OptionMenu(self, temp_model_params, *options)
        drop.place(x = 520, y = 470)
                   
        global puissance
        puis = ttk.Label(self, text = "Entrer la puissance maximal du projet").place(x = 250, y = 520)
        puissance = ttk.Entry(self)
        puissance.place(x = 520, y = 520)
        


        def predetermined():
            StartPage.num = 50
            StartPage.intervalDeg = 68.2
            StartPage.intervalEnc = 95
            
            try:
                enregistrer
            except NameError:
                pass
            else:
                enregistrer.place_forget()
            
            try:
                number
            except NameError:
                pass
            else:
                number.place_forget()
            
            try:
                histograme
            except NameError:
                pass
            else:
                histograme.place_forget()

            try:
                inter
            except NameError:
                pass
            else:
                inter.place_forget()
            try:
                confiance
            except NameError:
                pass
            else:
                confiance.place_forget()

            try:
                inter2
            except NameError:
                pass
            else:
                inter2.place_forget()
            try:
                confiance2
            except NameError:
                pass
            else:
                confiance2.place_forget()
        
        def register():
            StartPage.num = number.get()
            StartPage.intervalDeg = inter.get()
            StartPage.intervalEnc = inter2.get()
            print(StartPage.num)
            print(StartPage.intervalEnc)
            print(StartPage.intervalDeg)
            print("success")
            
                     
        def manual():
            
            global number
            global histograme
            histograme = ttk.Label(self, text = "Entrer le bins pour le calcul d'encrassement")
            histograme.place(x = 665, y = 70)
            number = ttk.Entry(self)
            number.place(x = 920, y = 70)
           
            global inter
            global confiance
            confiance = ttk.Label(self, text = "Entrer l'interval de confiance degradation")
            confiance.place(x = 665, y = 120)
            inter = ttk.Entry(self)
            inter.place(x = 920, y = 120)
           
            global inter2
            global confiance2
            confiance2 = ttk.Label(self, text = "Entrer l'interval de confiance d'encrassement")
            confiance2.place(x = 665, y = 170)
            inter2 = ttk.Entry(self)
            inter2.place(x = 920, y = 170)
            
            global enregistrer
            enregistrer = ttk.Button(self, text = "Enregistrer", width = 10, command = register)
            enregistrer.place(x = 1070, y = 165)
            
            return 
                        
        def test2(choice):
            choice = clicked.get()
            if choice == "predetermined":

                predetermined()
                
            elif choice == "manual":

                manual()
                
        options2 = [
            "predetermined",
            "manual"
        ]
        
        clicked = StringVar()
        clicked.set("Bonus options")
        
        drop2 = OptionMenu(self, clicked, *options2, command = test2)
        drop2.place(x = 1070, y = 65)
        

            
        def begin():
            print(StartPage.num)
            print(StartPage.intervalEnc)
            print(StartPage.intervalDeg) 
            choice = clicked.get()
            print(temp_model_params.get())
            print(filepath.get())
            
            if StartPage.filepathtest == "":
                StartPage.filepathtest = filepath.get()
            
            if StartPage.filepathtest != filepath.get():
                StartPage.conditional = 0
                StartPage.filepathtest = filepath.get()
            
            print(StartPage.filepathtest)
            
            value = 0
               
            if filepath.get() == "" or latitude.get() == "" or longitude.get() == "" or timezone.get() == "" \
                or gamma_pdc.get() == "" or azimuth.get() == "" or tilt.get() == "" or power_dc_rated.get() == "" \
                    or puissance.get() == "" or temp_model_params.get() == "Options" or StartPage.num == 0 \
                        or StartPage.intervalEnc == 0 or StartPage.intervalDeg == 0:
                erreurs(7, self)
                erreurs(4, self)
            else:      
                    
                if choice == "manual":

                    
                    try:
                        int(StartPage.num)
                    except ValueError:
                        erreurs(18, self)
                        return
                    else:
                        pass

                    try:
                        float(StartPage.intervalEnc)
                    except ValueError:
                        erreurs(1, self)
                        erreurs(2, self)
                        erreurs(19, self)
                        return
                    else:
                        pass

                    try:
                        float(StartPage.intervalDeg)
                    except ValueError:
                        erreurs(1, self)
                        erreurs(2, self)
                        erreurs(20, self)
                        return
                    else:
                        pass                        
#                 elif choice == "predetermined":
                
                try:
                    float(latitude.get())
                except ValueError:
                    erreurs(9, self)
                    return
                else:
                    pass
                
                try:
                    str(timezone.get())
                except ValueError:
                    erreurs(11, self)
                    return
                else:
                    pass
                
                try:
                    float(longitude.get())
                except ValueError:
                    erreurs(10, self)
                    return
                else:
                    pass
                
                try:
                    float(gamma_pdc.get())
                except ValueError:
                    erreurs(12, self)
                    return
                else:
                    pass
                
                try:
                    float(azimuth.get())
                except ValueError:
                    erreurs(13, self)
                    return
                else:
                    pass
                
                try:
                    float(tilt.get())
                except ValueError:
                    erreurs(14, self)
                    return
                else:
                    pass
                
                try:
                    float(power_dc_rated.get())
                except ValueError:
                    erreurs(16, self)
                    return
                else:
                    pass
                
                try:
                    float(puissance.get())
                except ValueError:
                    erreurs(17, self)
                    return
                else:
                    pass
                
                    
                if ".xlsx" not in filepath.get():
                    erreurs(6, self)
                    return

                elif "," in latitude.get():
                    erreurs(1, self)
                    erreurs(2, self)
                    return
                    
                elif "," in longitude.get():
                    erreurs(1, self)
                    erreurs(2, self)
                    return
                
                elif not isinstance(timezone.get(), str):
                    erreurs(11, self)
                    return
                    
                elif "GMT" not in timezone.get():
                        erreurs(5, self)
                        erreurs(8, self)
                        return
                        
                elif "," in gamma_pdc.get():
                    erreurs(1, self)
                    erreurs(2, self)
                    return
                
                elif "°" in azimuth.get():
                    erreurs(1, self)
                    erreurs(3, self)
                    return
                
                elif "°" in tilt.get():
                    erreurs(1, self)
                    erreurs(3, self)
                    return
                
                else:
                    value = 1
                    button1 = ttk.Button(self, text ="Importer les donnees",
                                       command = lambda : controller.show_frame(Page1))
                    button1.place(x = 5, y = 120)

                    button2 = ttk.Button(self, text ="StartPage",
                                        command = lambda : controller.show_frame(StartPage))
                    button2.place(x = 5, y = 70)

                    button3 = ttk.Button(self, text ="Normalisation",
                                        command = lambda : controller.show_frame(Page2))
                    button3.place(x = 5, y = 170)

                    button4 = ttk.Button(self, text ="Filtrage",
                                        command = lambda : controller.show_frame(Page3))
                    button4.place(x = 5, y = 220)

                    button5 = ttk.Button(self, text ="Rassemblement",
                                        command = lambda : controller.show_frame(Page4))
                    button5.place(x = 5, y = 270)

                    button6 = ttk.Button(self, text ="Calcul de la degradation",
                                        command = lambda : controller.show_frame(Page5))
                    button6.place(x = 5, y = 320)

                    button7 = ttk.Button(self, text ="Calcul d'encrassement",
                                        command = lambda : controller.show_frame(Page6))
                    button7.place(x = 5, y = 370)

        label1 = ttk.Label(self, text = "", textvariable = messages).place(x = 820, y = 400)

        ttk.Button(self, text = "Begin", width = 10, command = begin) \
            .place(x = 1070, y = 115)

# second window frame page1

class Page1(tk.Frame):
     
    def __init__(self, parent, controller):
         
        tk.Frame.__init__(self, parent)
        
        label = ttk.Label(self, text ="Importer les donnees", font = LARGEFONT)
        label.grid(row = 0, column = 0, sticky = tk.W, pady = 10)
  
        # button to show frame 2 with text
        # layout2
        
        button1 = ttk.Button(self, text ="Importer les donnees",
                           command = lambda : controller.show_frame(Page1))
        button1.place(x = 5, y = 120)

        button2 = ttk.Button(self, text ="StartPage",
                            command = lambda : controller.show_frame(StartPage))
        button2.place(x = 5, y = 70)
  
        button3 = ttk.Button(self, text ="Normalisation",
                            command = lambda : controller.show_frame(Page2))
        button3.place(x = 5, y = 170)

        button4 = ttk.Button(self, text ="Filtrage",
                            command = lambda : controller.show_frame(Page3))
        button4.place(x = 5, y = 220)
        
        button5 = ttk.Button(self, text ="Rassemblement",
                            command = lambda : controller.show_frame(Page4))
        button5.place(x = 5, y = 270)
  
        button6 = ttk.Button(self, text ="Calcul de la degradation",
                            command = lambda : controller.show_frame(Page5))
        button6.place(x = 5, y = 320)

        button7 = ttk.Button(self, text ="Calcul d'encrassement",
                            command = lambda : controller.show_frame(Page6))
        button7.place(x = 5, y = 370)
        
        def begin():
            print(StartPage.conditional)
            if len(filepath.get()) != 0 and StartPage.conditional >= 0:
                if StartPage.conditional == 0:
                    StartPage.conditional = StartPage.conditional + 1
                Importer(self, parent, controller, StartPage.conditional)
                
            else:
                erreurs(15, self)
        
        ttk.Button(self, text = "Begin", width = 10, command = begin).place(x = 360, y = 20)
        
# Third window frame page2 - Normalisation

class Page2(tk.Frame):
    
    def __init__(self, parent, controller):
        
        tk.Frame.__init__(self, parent)
        
        label = ttk.Label(self, text ="Normalisation", font = LARGEFONT)
        label.grid(row = 0, column = 0, sticky = tk.W, pady = 10)
  
        button1 = ttk.Button(self, text ="Importer les donnees",
                           command = lambda : controller.show_frame(Page1))
        button1.place(x = 5, y = 120)

        button2 = ttk.Button(self, text ="StartPage",
                            command = lambda : controller.show_frame(StartPage))
        button2.place(x = 5, y = 70)
  
        button3 = ttk.Button(self, text ="Normalisation",
                            command = lambda : controller.show_frame(Page2))
        button3.place(x = 5, y = 170)

        button4 = ttk.Button(self, text ="Filtrage",
                            command = lambda : controller.show_frame(Page3))
        button4.place(x = 5, y = 220)
        
        button5 = ttk.Button(self, text ="Rassemblement",
                            command = lambda : controller.show_frame(Page4))
        button5.place(x = 5, y = 270)
  
        button6 = ttk.Button(self, text ="Calcul de la degradation",
                            command = lambda : controller.show_frame(Page5))
        button6.place(x = 5, y = 320)

        button7 = ttk.Button(self, text ="Calcul d'encrassement",
                            command = lambda : controller.show_frame(Page6))
        button7.place(x = 5, y = 370)
        
        def begin():
            print(StartPage.conditional)
            if len(filepath.get()) != 0 and StartPage.conditional >= 1:
                if StartPage.conditional == 1:
                    StartPage.conditional = StartPage.conditional + 1
                Normalisation(self, parent, controller, StartPage.conditional)

            else:
                erreurs(15, self)
                
        ttk.Button(self, text = "Begin", width = 10, command = begin).place(x = 360, y = 20)
        
# Fourth window frame page3 - Filtrage
        
class Page3(tk.Frame):
    
    def __init__(self, parent, controller):
        
        tk.Frame.__init__(self, parent)
        
        label = ttk.Label(self, text ="Filtrage", font = LARGEFONT)
        label.grid(row = 0, column = 0, sticky = tk.W, pady = 10)
  
        button1 = ttk.Button(self, text ="Importer les donnees",
                           command = lambda : controller.show_frame(Page1))
        button1.place(x = 5, y = 120)

        button2 = ttk.Button(self, text ="StartPage",
                            command = lambda : controller.show_frame(StartPage))
        button2.place(x = 5, y = 70)
  
        button3 = ttk.Button(self, text ="Normalisation",
                            command = lambda : controller.show_frame(Page2))
        button3.place(x = 5, y = 170)

        button4 = ttk.Button(self, text ="Filtrage",
                            command = lambda : controller.show_frame(Page3))
        button4.place(x = 5, y = 220)
        
        button5 = ttk.Button(self, text ="Rassemblement",
                            command = lambda : controller.show_frame(Page4))
        button5.place(x = 5, y = 270)
  
        button6 = ttk.Button(self, text ="Calcul de la degradation",
                            command = lambda : controller.show_frame(Page5))
        button6.place(x = 5, y = 320)

        button7 = ttk.Button(self, text ="Calcul d'encrassement",
                            command = lambda : controller.show_frame(Page6))
        button7.place(x = 5, y = 370)
        
        def begin():
            print(StartPage.conditional)
            if len(filepath.get()) != 0 and StartPage.conditional >= 2:
                if StartPage.conditional == 2:
                    StartPage.conditional = StartPage.conditional + 1                
                Filtrage(self, parent, controller, StartPage.conditional)

            else:
                erreurs(15, self)
                
        ttk.Button(self, text = "Begin", width = 10, command = begin).place(x = 360, y = 20)
        
# Fifth window frame page4 - Rassemblement
        
class Page4(tk.Frame):
    
    def __init__(self, parent, controller):
        
        tk.Frame.__init__(self, parent)
        
        label = ttk.Label(self, text ="Rassemblement", font = LARGEFONT)
        label.grid(row = 0, column = 0, sticky = tk.W, pady = 10)
  
        button1 = ttk.Button(self, text ="Importer les donnees",
                           command = lambda : controller.show_frame(Page1))
        button1.place(x = 5, y = 120)

        button2 = ttk.Button(self, text ="StartPage",
                            command = lambda : controller.show_frame(StartPage))
        button2.place(x = 5, y = 70)
  
        button3 = ttk.Button(self, text ="Normalisation",
                            command = lambda : controller.show_frame(Page2))
        button3.place(x = 5, y = 170)

        button4 = ttk.Button(self, text ="Filtrage",
                            command = lambda : controller.show_frame(Page3))
        button4.place(x = 5, y = 220)
        
        button5 = ttk.Button(self, text ="Rassemblement",
                            command = lambda : controller.show_frame(Page4))
        button5.place(x = 5, y = 270)
  
        button6 = ttk.Button(self, text ="Calcul de la degradation",
                            command = lambda : controller.show_frame(Page5))
        button6.place(x = 5, y = 320)

        button7 = ttk.Button(self, text ="Calcul d'encrassement",
                            command = lambda : controller.show_frame(Page6))
        button7.place(x = 5, y = 370)
        
        def begin():
            print(StartPage.conditional)
            if len(filepath.get()) != 0 and StartPage.conditional >= 3:
                if StartPage.conditional == 3:
                    StartPage.conditional = StartPage.conditional + 1                
                Rassemblement(self, parent, controller, StartPage.conditional)
                
            else:
                erreurs(15, self)
            
        ttk.Button(self, text = "Begin", width = 10, command = begin).place(x = 360, y = 20)
        
# Sixth window frame page5 - Calcul de la degradation
        
class Page5(tk.Frame):
    
    def __init__(self, parent, controller):
        
        tk.Frame.__init__(self, parent)
        
        label = ttk.Label(self, text ="Calcul de la degradation", font = LARGEFONT)
        label.grid(row = 0, column = 0, sticky = tk.W, pady = 10)
  
        button1 = ttk.Button(self, text ="Importer les donnees",
                           command = lambda : controller.show_frame(Page1))
        button1.place(x = 5, y = 120)

        button2 = ttk.Button(self, text ="StartPage",
                            command = lambda : controller.show_frame(StartPage))
        button2.place(x = 5, y = 70)
  
        button3 = ttk.Button(self, text ="Normalisation",
                            command = lambda : controller.show_frame(Page2))
        button3.place(x = 5, y = 170)

        button4 = ttk.Button(self, text ="Filtrage",
                            command = lambda : controller.show_frame(Page3))
        button4.place(x = 5, y = 220)
        
        button5 = ttk.Button(self, text ="Rassemblement",
                            command = lambda : controller.show_frame(Page4))
        button5.place(x = 5, y = 270)
  
        button6 = ttk.Button(self, text ="Calcul de la degradation",
                            command = lambda : controller.show_frame(Page5))
        button6.place(x = 5, y = 320)

        button7 = ttk.Button(self, text ="Calcul d'encrassement",
                            command = lambda : controller.show_frame(Page6))
        button7.place(x = 5, y = 370)
        
        def begin():
            print(StartPage.conditional)
            if len(filepath.get()) != 0 and StartPage.conditional >= 4:
                if StartPage.conditional == 4:
                    StartPage.conditional = StartPage.conditional + 1                
                Degradation(self, parent, controller, StartPage.conditional, StartPage.intervalDeg)
                
            else:
                erreurs(15, self)
            
        ttk.Button(self, text = "Begin", width = 10, command = begin).place(x = 360, y = 20)
        
# Seventh window frame page6 - Calcul d'encrassement
        
class Page6(tk.Frame):
    
    def __init__(self, parent, controller):
        
        tk.Frame.__init__(self, parent)
        
        label = ttk.Label(self, text ="Calcul d'encrassement", font = LARGEFONT)
        label.grid(row = 0, column = 0, sticky = tk.W, pady = 10)
  
        button1 = ttk.Button(self, text ="Importer les donnees",
                           command = lambda : controller.show_frame(Page1))
        button1.place(x = 5, y = 120)

        button2 = ttk.Button(self, text ="StartPage",
                            command = lambda : controller.show_frame(StartPage))
        button2.place(x = 5, y = 70)
  
        button3 = ttk.Button(self, text ="Normalisation",
                            command = lambda : controller.show_frame(Page2))
        button3.place(x = 5, y = 170)

        button4 = ttk.Button(self, text ="Filtrage",
                            command = lambda : controller.show_frame(Page3))
        button4.place(x = 5, y = 220)
        
        button5 = ttk.Button(self, text ="Rassemblement",
                            command = lambda : controller.show_frame(Page4))
        button5.place(x = 5, y = 270)
  
        button6 = ttk.Button(self, text ="Calcul de la degradation",
                            command = lambda : controller.show_frame(Page5))
        button6.place(x = 5, y = 320)

        button7 = ttk.Button(self, text ="Calcul d'encrassement",
                            command = lambda : controller.show_frame(Page6))
        button7.place(x = 5, y = 370)
        
        def begin():
            print(StartPage.conditional)
            if len(filepath.get()) != 0 and StartPage.conditional >= 5:
                if StartPage.conditional == 5:
                    StartPage.conditional = StartPage.conditional + 1                
                Encrassement(self, parent, controller, Page6, StartPage.conditional, StartPage.num, StartPage.intervalEnc)
            else:
                erreurs(15, self)
            
        ttk.Button(self, text = "Begin", width = 10, command = begin).place(x = 360, y = 20)
        

# Driver Code


app = tkinterApp()
app.mainloop()